In [1]:
import pandas as pd
import matplotlib

In [2]:
input_file = 'BankChurners.csv'
cc_data = pd.read_csv(input_file)
cc_data.head(5).style.hide_index()

CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.000000,777,11914.000000,1.335000,1144,42,1.625000,0.061000,0.000093,0.999910
818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.000000,864,7392.000000,1.541000,1291,33,3.714000,0.105000,0.000057,0.999940
713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.000000,0,3418.000000,2.594000,1887,20,2.333000,0.000000,0.000021,0.999980
769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,3,4,1,3313.000000,2517,796.000000,1.405000,1171,20,2.333000,0.760000,0.000134,0.999870
709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.000000,0,4716.000000,2.175000,816,28,2.500000,0.000000,0.000022,0.999980


In [3]:
# The last two columns had names that are too long.  We're going to shorten them to nb1 and nb2 for easier readability.
#  Changing the column names from an index to a list to ease modifications.
column_names = list(cc_data.columns)

# We want to keep the detailed information from the original long names.
nb1_label = column_names[-2]
nb2_label = column_names[-1]

#  Replacing the long names with nb1 and nb2
column_names[-2] = 'nb1'
column_names[-1] = 'nb2'

#  Now renaming the columns on the dataframe itself.
cc_data.columns = pd.Index(column_names)

# Dropping CLIENTNUM as it has no predictive power.  (client 1 is not expected to be systematically different than client 100)
cc_data.drop(columns=['CLIENTNUM'], inplace = True)

# Taking a peek.
cc_data.head(5).style.hide_index()

Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,nb1,nb2
Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.000000,777,11914.000000,1.335000,1144,42,1.625000,0.061000,0.000093,0.999910
Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.000000,864,7392.000000,1.541000,1291,33,3.714000,0.105000,0.000057,0.999940
Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.000000,0,3418.000000,2.594000,1887,20,2.333000,0.000000,0.000021,0.999980
Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,3,4,1,3313.000000,2517,796.000000,1.405000,1171,20,2.333000,0.760000,0.000134,0.999870
Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.000000,0,4716.000000,2.175000,816,28,2.500000,0.000000,0.000022,0.999980


We want to look at the distinct values of each string column (dtype = object).

In [4]:
text_columns = list(cc_data.select_dtypes(include='object').columns)

for col_name in text_columns:
    uniq_col_vals = list(cc_data[col_name].unique())
    num_uniq_col_vals = str(len(uniq_col_vals))
    print("\nThere are " +
          num_uniq_col_vals +
          " number unique values in " +
          col_name +
          ". They are:\n")
    for i in range(int(num_uniq_col_vals)):
        print(uniq_col_vals[i])


There are 2 number unique values in Attrition_Flag. They are:

Existing Customer
Attrited Customer

There are 2 number unique values in Gender. They are:

M
F

There are 7 number unique values in Education_Level. They are:

High School
Graduate
Uneducated
Unknown
College
Post-Graduate
Doctorate

There are 4 number unique values in Marital_Status. They are:

Married
Single
Unknown
Divorced

There are 6 number unique values in Income_Category. They are:

$60K - $80K
Less than $40K
$80K - $120K
$40K - $60K
$120K +
Unknown

There are 4 number unique values in Card_Category. They are:

Blue
Gold
Silver
Platinum


In [16]:
cc_data.columns

Index(['Customer_Age', 'Dependent_count', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio', 'nb1',
       'nb2', 'Attrition_Flag_Existing Customer_ind', 'Gender_M_ind',
       'Education_Level_Doctorate_ind', 'Education_Level_Graduate_ind',
       'Education_Level_High School_ind', 'Education_Level_Post-Graduate_ind',
       'Education_Level_Uneducated_ind', 'Education_Level_Unknown_ind',
       'Marital_Status_Married_ind', 'Marital_Status_Single_ind',
       'Marital_Status_Unknown_ind', 'Income_Category_$40K - $60K_ind',
       'Income_Category_$60K - $80K_ind', 'Income_Category_$80K - $120K_ind',
       'Income_Category_Less than $40K_ind', 'Income_Category_Unknown_ind',
       'Card_Category_Gold_ind', 'Card_Category_Platinum_ind',
       'Car

In [5]:
# Checking to see if we have any null data in any columns.
print("\nCount total NaN at each column in a DataFrame:\n\n", 
      cc_data.isnull().sum())
# We do not, so we don't need to do any dropping or any mean imputation.


Count total NaN at each column in a DataFrame:

 Attrition_Flag              0
Customer_Age                0
Gender                      0
Dependent_count             0
Education_Level             0
Marital_Status              0
Income_Category             0
Card_Category               0
Months_on_book              0
Total_Relationship_Count    0
Months_Inactive_12_mon      0
Contacts_Count_12_mon       0
Credit_Limit                0
Total_Revolving_Bal         0
Avg_Open_To_Buy             0
Total_Amt_Chng_Q4_Q1        0
Total_Trans_Amt             0
Total_Trans_Ct              0
Total_Ct_Chng_Q4_Q1         0
Avg_Utilization_Ratio       0
nb1                         0
nb2                         0
dtype: int64


In [6]:
def dummy_making_process(my_col, my_df = cc_data):
    tmp_dummy_variable = pd.get_dummies(my_df[my_col])
    prefix = my_col
    
    # Starting on the second variable to avoid the dummy variable trap.
    middle = list(tmp_dummy_variable.columns)[1:]
    suffix = "ind"
    index = 0
    while index < len(middle):
        new_name = prefix + "_" + middle[index] + "_" + suffix
        my_df[new_name] = tmp_dummy_variable[middle[index]]
        index = index + 1

def archive_and_delete_dummied_col(my_col, my_df = cc_data):
    try:
        now_dummied_columns[my_col] = my_df[my_col]
    except:
        now_dummied_columns = pd.DataFrame(my_df[my_col])
    my_df.drop(columns = my_col, inplace = True)

In [7]:
# Taking a look before making the dummy variables.
cc_data.head(3).style.hide_index()

Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,nb1,nb2
Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.000000,777,11914.000000,1.335000,1144,42,1.625000,0.061000,0.000093,0.999910
Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.000000,864,7392.000000,1.541000,1291,33,3.714000,0.105000,0.000057,0.999940
Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.000000,0,3418.000000,2.594000,1887,20,2.333000,0.000000,0.000021,0.999980


In [9]:
# Creating dummies and deleting columns that have been made into dummy variable columns.
for col in text_columns:
    dummy_making_process(my_col = col)
    archive_and_delete_dummied_col(my_col = col)

In [10]:
cc_data.head(3).style.hide_index()

Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,nb1,nb2,Attrition_Flag_Existing Customer_ind,Gender_M_ind,Education_Level_Doctorate_ind,Education_Level_Graduate_ind,Education_Level_High School_ind,Education_Level_Post-Graduate_ind,Education_Level_Uneducated_ind,Education_Level_Unknown_ind,Marital_Status_Married_ind,Marital_Status_Single_ind,Marital_Status_Unknown_ind,Income_Category_$40K - $60K_ind,Income_Category_$60K - $80K_ind,Income_Category_$80K - $120K_ind,Income_Category_Less than $40K_ind,Income_Category_Unknown_ind,Card_Category_Gold_ind,Card_Category_Platinum_ind,Card_Category_Silver_ind
45,3,39,5,1,3,12691.000000,777,11914.000000,1.335000,1144,42,1.625000,0.061000,0.000093,0.999910,1,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0
49,5,44,6,1,2,8256.000000,864,7392.000000,1.541000,1291,33,3.714000,0.105000,0.000057,0.999940,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
51,3,36,4,1,0,3418.000000,0,3418.000000,2.594000,1887,20,2.333000,0.000000,0.000021,0.999980,1,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0


In [41]:
# Eventually, I want to get our data to this point:
#  https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
X, y = load_iris(return_X_y=True)
clf = LogisticRegression(random_state=0).fit(X, y)
clf.predict(X[:2, :])
clf.predict_proba(X[:2, :])
clf.score(X, y)

C:\Users\Owner\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9733333333333334

In [42]:
#  Currently, the sklearn logistic regression does not converge on the unscaled data.
from sklearn.linear_model import LogisticRegression

y_col = 'Attrition_Flag_Existing Customer_ind'
X = cc_data.drop(columns = (y_col))
y = cc_data[y_col]

try:
    logit_model = LogisticRegression(random_state = 0).fit(X, y)
except:
    print("Looks like I need to do more pre-processing of the data.")

C:\Users\Owner\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [44]:
# Doing some data pre-processing.
from sklearn.preprocessing import StandardScaler
X = cc_data.drop(columns = (y_col))
y = cc_data[y_col]

X_scaled = StandardScaler().fit(X).transform(X)
logit_model = LogisticRegression(random_state = 0).fit(X_scaled, y)

In [45]:
logit_model.predict(X_scaled[:2, :])

array([1, 1], dtype=uint8)

In [47]:
logit_model.predict_proba(X_scaled[:2, :])

array([[5.37954255e-05, 9.99946205e-01],
       [8.72082498e-06, 9.99991279e-01]])

In [48]:
logit_model.score(X_scaled, y)

1.0

In [ ]:
# Need to work on doing some cross validation folds or at least a single test-train split to really see how accurate this is.